# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686599


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:40,  1.44s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:47,  1.76s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:44,  1.69s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:29,  1.21s/it]

...

Rendering models:  73%|███████▎  | 22/30 [00:08<00:01,  6.81it/s]

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  7.46it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  7.76it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  9.02it/s]

not-logged-in-6c3c85931d83a1164a0d    0.001986
beckyrother                           0.019948
not-logged-in-20a275b53abec8db9f41    0.011219
not-logged-in-bc5fdb9ea8d611e60b8b    0.000233
shirogane                             0.000201
Camelron                              0.000196
not-logged-in-82c06f1bd03315127bbc    0.010632
austingasbarra                        0.000197
not-logged-in-c3e2bb63f9ec3bdeea91    0.000326
mtajnai                               0.008634
not-logged-in-f09819078c043556a4da    0.003468
Alexatwin                             0.000388
not-logged-in-5a8bb7be433678ade709    0.000413
not-logged-in-258b74099ecf199aff36    0.001929
Koro132                               0.000750
lumpofgiggles                         0.000174
ShayDouthi                            0.104518
pangeli5                              0.000650
LunaMoona                             0.002604
RaulE                                 0.000375
EM4893                                0.000372
BraxtonKinney

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())